In [ ]:
!pip install sparkdl keras tensorflow==1.13.1 tensorframes==0.2.7 tensorflowonspark==1.3.0 jieba

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from sparkdl import readImages
import tensorflow as tf
import tensorframes as tfs

Using TensorFlow backend.
/Users/aleksandrsavchenko/opt/anaconda3/envs/stepik-ds-course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/aleksandrsavchenko/opt/anaconda3/envs/stepik-ds-course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/aleksandrsavchenko/opt/anaconda3/envs/stepik-ds-course/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(

In [4]:
spark = SparkSession.builder \
        .appName("PySparkCatDogJob") \
        .config("spark.jars", "*.jar") \
        .config("packages","databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11,databricks:tensorframes:0.6.0-s_2.11") \
        .getOrCreate()

sc = spark.sparkContext
sc.PACKAGE_EXTENSIONS=("databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11,databricks:tensorframes:0.6.0-s_2.11")

In [5]:
spark

In [6]:
from pyspark.sql.functions import lit

In [7]:
cat_df = readImages("cat_dog/cat").withColumn("label", lit(0))

In [8]:
dog_df = readImages("cat_dog/dog").withColumn("label", lit(1))

In [9]:
cat_df.printSchema()

root
 |-- filePath: string (nullable = false)
 |-- image: struct (nullable = true)
 |    |-- mode: string (nullable = false)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- data: binary (nullable = false)
 |-- label: integer (nullable = false)



In [10]:
(cat_df_train, cat_df_test) = cat_df.randomSplit([0.7, 0.3])
(dog_df_train, dog_df_test) = dog_df.randomSplit([0.7, 0.3])

In [11]:
df_train = cat_df_train.union(dog_df_train)

In [12]:
df_test = cat_df_test.union(dog_df_test)

In [16]:
df_train.count()

216

In [17]:
df_test.count()

84

In [15]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=1, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr]) 
p_model = p.fit(df_train)

Instructions for updating:
Colocations handled automatically by placer.


2019-12-09 22:53:22,898 WARNING (MainThread-99959) From /Users/aleksandrsavchenko/opt/anaconda3/envs/stepik-ds-course/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants


2019-12-09 22:53:31,085 WARNING (MainThread-99959) From /Users/aleksandrsavchenko/opt/anaconda3/envs/stepik-ds-course/lib/python3.7/site-packages/sparkdl/graph/utils.py:189: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants


Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph


...

2019-12-09 22:53:49,286 WARNING (MainThread-99959) From /Users/aleksandrsavchenko/opt/anaconda3/envs/stepik-ds-course/lib/python3.7/site-packages/sparkdl/transformers/tf_image.py:178: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


INFO:tensorflow:Froze 0 variables.


2019-12-09 22:53:50,284 INFO (MainThread-99959) Froze 0 variables.


INFO:tensorflow:Converted 0 variables to const ops.


2019-12-09 22:53:50,401 INFO (MainThread-99959) Converted 0 variables to const ops.
2019-12-09 22:53:50,983 INFO (MainThread-99959) Fetch names: ['sdl_flattened_mixed10/concat:0']
2019-12-09 22:53:50,984 INFO (MainThread-99959) Spark context = <SparkContext master=local[*] appName=PySparkCatDogJob>


In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator 
tested_df = p_model.transform(df_test)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy") 
print("Test set accuracy = " + str(evaluator.evaluate(tested_df.select("prediction", "label"))))

Test set accuracy = 0.78756454354


In [19]:
spark.stop()